In [110]:
from ipywidgets import *
import numpy as np
from scipy.spatial.transform import Rotation as R
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from IPython.display import display,clear_output
import time
style = {'description_width':'initial'}
style2 = {'value_width':'initial'}
input_lyt = Layout(width='400px',height='130px')
update_lyt = Layout(width='80px')

################################################################################
def text(num):
    return str(round(num,4))
def dcm_pretty(vrot):
    dcm = vrot.rot.as_dcm()
    text_pretty = r'\(M = \begin{bmatrix} '
    for row in dcm:
        for col in row:
            text_pretty += text(col) + ' & '
        text_pretty = text_pretty[:-3] + ' \\\ '
    text_pretty = text_pretty[:-3] + '\end{bmatrix}\)'
    return text_pretty
def dcm_code(vrot):
    dcm = vrot.rot.as_dcm()    
    text_code = '[ '
    for row in dcm:
        text_code += '[ '
        for col in row:
            text_code += text(col) + ', '
        text_code = text_code[:-2] + ' ] '
    text_code += ' ]'
    return text_code
def euler_pretty(vrot):
    euler = vrot.rot.as_euler('xyz',degrees=True)
    text_pretty = '(x,y,z) = ('
    for entry in euler:
        text_pretty += text(entry) + ', '
    text_pretty = text_pretty[:-2] + ')'
    return text_pretty
def euler_code(vrot):
    euler = vrot.rot.as_euler('xyz',degrees=True)
    text_code = '[ '
    for entry in euler:
        text_code += text(entry) + ', '
    text_code = text_code[:-2] + ' ]'
    return text_code
def quat_pretty(vrot):
    quat = vrot.rot.as_quat()
    text_pretty = '(x,y,z,w) = ('
    for entry in quat:
        text_pretty += text(entry) + ', '
    text_pretty = text_pretty[:-2] + ')'
    return text_pretty
def quat_code(vrot):
    quat = vrot.rot.as_quat()
    text_code = '[ '
    for entry in quat:
        text_code += text(entry) + ', '
    text_code = text_code[:-2] + ' ]'
    return text_code
def axis_angle_pretty(vrot):
    axis_angle = vrot.rot.as_quat() 
    text_pretty = '[ '
    for entry in axis_angle:
        text_pretty += text(entry) + ', '
    text_pretty = text_pretty[:-2] + ']'
    return text_pretty
def axis_angle_code(vrot):
    axis_angle = vrot.rot.as_quat()
    text_code = '[ '
    for entry in axis_angle:
        text_code += text(entry) + ', '
    text_code = text_code[:-2] + ']'
    return text_code

################################################################################
class ViewableRot:
    def __init__(self, rot=R.from_rotvec([0,0,0])):
        self.rot = rot
        self.is_valid = widgets.Valid(value=True)
        
        self.use_testpoint = True
        self.use_shadowbox = True

        self.default_testpoint = [-5,5,5]
        self.default_graphview = [30,45]

        self.testpoint = self.default_testpoint
        self.graphview = self.default_graphview
    def get_rot(self):
        return self.rot
    def set_rot(self, rot):
        self.rot = rot
        self.set_is_valid(False)
    def set_is_valid(self, is_valid):
        self.is_valid.value = is_valid
    def set_use_testpoint(self, use_testpoint):
        self.use_testpoint = use_testpoint
        self.set_is_valid(False)
    def set_use_shadowbox(self, use_shadowbox):
        self.use_shadowbox = use_shadowbox
        self.set_is_valid(False)
    def set_default_testpoint(self, default_testpoint):
        self.default_testpoint = default_testpoint
        self.set_is_valid(False)
    def set_default_graphview(self, default_graphview):
        self.default_graphview = default_graphview
        self.set_is_valid(False)
    def set_testpoint(self, testpoint):
        self.testpoint = testpoint
        self.set_is_valid(False)
    def set_graphview(self, graphview):
        self.graphview = graphview
        self.set_is_valid(False)
        
    def as_dcm(self):
        pretty = widgets.Label(value=dcm_pretty(self), layout = Layout(height='100px'))
        code = widgets.Label(value=dcm_code(self))
        return widgets.VBox([pretty,code])
    def as_euler(self, orientation = 'xyz', use_degrees = True):
        pretty = widgets.Label(value=euler_pretty(self))
        code = widgets.Label(value=euler_code(self))
        return widgets.VBox([pretty,code])
    def as_quat(self):
        pretty = widgets.Label(value=quat_pretty(self))
        code = widgets.Label(value=quat_code(self))
        return widgets.VBox([pretty,code])
    def as_axis_angle(self):
        pretty = widgets.Label(value=axis_angle_pretty(self))
        code = widgets.Label(value=axis_angle_code(self))
        return widgets.VBox([pretty,code])
    def display(self):
        WSelect = widgets.RadioButtons(style=style, disabled=False,
                description='Rotation Type', options=['DCM', 'Axis-Angle', 'Euler Angles', 'Quaternion'])
        WViewInit = self.as_dcm()
        items = [WSelect, WViewInit]
        w = widgets.VBox(items)

        def refresh_display(sender):
            self.is_valid.value=True
            if (WSelect.value == 'DCM'):
                WViewNew = self.as_dcm()
                w.children = [WSelect, WViewNew]
            if (WSelect.value == 'Axis-Angle'):
                WViewNew = self.as_axis_angle()
                w.children = [WSelect, WViewNew]
            if (WSelect.value == 'Euler Angles'):
                WViewNew = self.as_euler()
                w.children = [WSelect, WViewNew]
            if (WSelect.value == 'Quaternion'):
                WViewNew = self.as_quat()
                w.children = [WSelect, WViewNew]
        
        WSelect.observe(refresh_display)
        self.is_valid.observe(refresh_display)

        return w
    def input_dcm(self):
        current = self.rot.as_dcm()
        cellwidth = '75px'
        print(current[0][0])
        a11_input = widgets.BoundedFloatText(description='a11',value=current[0][0],style=style,layout=Layout(width=cellwidth))
        a12_input = widgets.BoundedFloatText(description='a12',value=current[0][1],style=style,layout=Layout(width=cellwidth))
        a13_input = widgets.BoundedFloatText(description='a13',value=current[0][2],style=style,layout=Layout(width=cellwidth))
        
        a21_input = widgets.BoundedFloatText(description='a21',value=current[1][0],style=style,layout=Layout(width=cellwidth))
        a22_input = widgets.BoundedFloatText(description='a22',value=current[1][1],style=style,layout=Layout(width=cellwidth))
        a23_input = widgets.BoundedFloatText(description='a23',value=current[1][2],style=style,layout=Layout(width=cellwidth))
        
        a31_input = widgets.BoundedFloatText(description='a31',value=current[2][0],style=style,layout=Layout(width=cellwidth))
        a32_input = widgets.BoundedFloatText(description='a32',value=current[2][1],style=style,layout=Layout(width=cellwidth))
        a33_input = widgets.BoundedFloatText(description='a33',value=current[2][2],style=style,layout=Layout(width=cellwidth))
        
        update_input = widgets.Button(description='Update',layout=update_lyt)
        
        dcm1_input = widgets.VBox([a11_input,a21_input,a31_input])
        dcm2_input = widgets.VBox([a12_input,a22_input,a32_input])
        dcm3_input = widgets.VBox([a13_input,a23_input,a33_input])
        dcm_input = widgets.HBox([dcm1_input,dcm2_input,dcm3_input,update_input],layout=input_lyt)
        def send_input(sender):
            dcm9 = [[a11_input.value,a12_input.value,a13_input.value],[a21_input.value,a22_input.value,a23_input.value],[a31_input.value,a32_input.value,a33_input.value]]
            rot=R.from_rotvec(dcm9)
            self.set_rot(rot)
        update_input.on_click(send_input)
        return dcm_input
    def input_axis_angle(self):
        x_input = widgets.BoundedFloatText(description='x',style=style,layout=Layout(width="60px"))
        y_input = widgets.BoundedFloatText(description='y',style=style,layout=Layout(width="60px"))
        z_input = widgets.BoundedFloatText(description='z',style=style,layout=Layout(width="60px"))
        theta_input = widgets.BoundedFloatText(description=chr(952),style=style,layout=Layout(width="60px"))
        update_input = widgets.Button(description='Update',layout=update_lyt)
        axis_angle_input = widgets.HBox([x_input,y_input,z_input,theta_input,update_input],layout=input_lyt)
        def send_input(sender):
            norm = np.linalg.norm(np.array([x_input.value, y_input.value, z_input.value]))
            axis_angle3 = [x_input.value*norm/theta_input.value, y_input.value*norm/theta_input.value, z_input.value*norm/theta_input.value]
            rot = R.from_rotvec(axis_angle3)
            self.set_rot(rot)
        update_input.on_click(send_input)
        return axis_angle_input
    def input_quat(self):
        x_input = widgets.BoundedFloatText(description='x',style=style,layout=Layout(width="60px"))
        y_input = widgets.BoundedFloatText(description='y',style=style,layout=Layout(width="60px"))
        z_input = widgets.BoundedFloatText(description='z',style=style,layout=Layout(width="60px"))
        w_input = widgets.BoundedFloatText(description='w', value=1,style=style,layout=Layout(width="60px"))
        update_input = widgets.Button(description='Update',layout=update_lyt)
        quad_input = widgets.HBox([x_input,y_input,z_input,w_input,update_input],layout=input_lyt)
        def send_input(sender):
            quat4 = [x_input.value, y_input.value, z_input.value, w_input.value]
            if (quat4 != [0,0,0,0]):
                rot = R.from_quat(quat4)
                self.set_rot(rot)
        update_input.on_click(send_input)
        return quad_input
    def input_euler(self):
        current = self.rot.as_euler('xyz',True)
        x_input = widgets.BoundedFloatText(description='x',value=current[0],min=-360,max=360,style=style,layout=Layout(width="60px"))
        y_input = widgets.BoundedFloatText(description='y',value=current[1],min=-360,max=360,style=style,layout=Layout(width="60px"))
        z_input = widgets.BoundedFloatText(description='z',value=current[2],min=-360,max=360,style=style,layout=Layout(width="60px"))
        o_input = widgets.Text(description='orientation', value='xyz',layout=Layout(width="130px"))
        update_input = widgets.Button(description='Update',layout=update_lyt)
        euler_input = widgets.HBox([x_input,y_input,z_input,o_input,update_input],layout=input_lyt)
        def send_input(sender):
            euler3 = [x_input.value, y_input.value, z_input.value]
            rot = R.from_euler(o_input.value, euler3, degrees=True)
            self.set_rot(rot)
        update_input.on_click(send_input)
        return euler_input
    def input_rot(self):
        WSelect = widgets.RadioButtons(style=style, disabled=False,
            description='Rotation Type', options=['Euler Angles', 'DCM', 'Axis-Angle', 'Quaternion'])
        WInputInit = self.input_euler()
        items = [WSelect, WInputInit]
        w = widgets.VBox(items)

        def refresh_input(sender):
            self.is_valid.value=True
            if (WSelect.value == 'DCM'):
                WInputNew = self.input_dcm()
                w.children = [WSelect, WInputNew]
            if (WSelect.value == 'Axis-Angle'):
                WInputNew = self.input_axis_angle()
                w.children = [WSelect, WInputNew]
            if (WSelect.value == 'Euler Angles'):
                WInputNew = self.input_euler()
                w.children = [WSelect, WInputNew]
            if (WSelect.value == 'Quaternion'):
                WInputNew = self.input_quat()
                w.children = [WSelect, WInputNew]
        
        WSelect.observe(refresh_input)
        self.is_valid.observe(refresh_input)

        return w
    def input_testpoint(self):
        x_input = widgets.BoundedFloatText(description='x',value=self.testpoint[0],min=-10,max=10,style=style,layout=Layout(width="60px"))
        y_input = widgets.BoundedFloatText(description='y',value=self.testpoint[1],min=-10,max=10,style=style,layout=Layout(width="60px"))
        z_input = widgets.BoundedFloatText(description='z',value=self.testpoint[2],min=-10,max=10,style=style,layout=Layout(width="60px"))
        testpoint_input = widgets.HBox([x_input,y_input,z_input])
        def send_input(sender):
            self.set_testpoint([x_input.value,y_input.value,z_input.value])
        x_input.observe(send_input)
        y_input.observe(send_input)
        z_input.observe(send_input)
        
        return testpoint_input
    def request_testpoint(self):
        WSelect = widgets.Checkbox(value=self.use_testpoint,description='Use testpoint',disabled=False)
        testpoint_input = self.input_testpoint()
        testpoint_request = widgets.VBox([WSelect,testpoint_input])
        def send_input(sender):
            self.set_use_testpoint(WSelect.value)
        WSelect.observe(send_input)
        return testpoint_request
    def graph(self): 
        out=widgets.Output()
        fig = plt.figure(figsize=(10,10))
        ax = plt.axes(projection='3d')
        with out:
            plt.show(ax.figure)
            
        btn_gph_lyt = Layout(width='60px')
        w_right = widgets.Button(description='right',layout=btn_gph_lyt)
        w_left = widgets.Button(description='left',layout=btn_gph_lyt)
        w_up = widgets.Button(description='up',layout=btn_gph_lyt)
        w_down = widgets.Button(description='down',layout=btn_gph_lyt)
        w_reset = widgets.Button(description='reset',layout=btn_gph_lyt)
        hbox = widgets.HBox([w_right,w_left,w_up,w_down,w_reset])
        testpoint_request = self.request_testpoint()
        #vbox = widgets.VBox([out,hbox]) # without testpoint
        vbox = widgets.VBox([testpoint_request,out,hbox]) # with testpoint
        vbox.layout.align_items='center'
        
        def click(el,az):
            az_angle = ax.azim + az
            el_angle = ax.elev + el
            ax.view_init(el_angle,az_angle)
            with out:
                clear_output(wait=True)
                display(ax.figure)
        rotamt = 15
        def left(sender):
            click(0,-rotamt)
        def right(sender):
            click(0,rotamt)
        def up(sender):
            click(rotamt,0)
        def down(sender):
            click(-rotamt,0)
        def reset(sender):
            ax.view_init(30,45)
            click(0,0)
        def refresh_graph(sender):
            with out:
                clear_output(wait=True)
                ax.cla()
                #axes
                ax.plot([-10,0], [0,0], [0,0], color='#cccccc')
                ax.plot([0,0], [-10,0], [0,0], color='#cccccc')
                ax.plot([0,0], [0,0], [-10,0], color='#cccccc')
                ax.plot([0,10], [0,0], [0,0], color='#eebbbb')
                ax.plot([0,0], [0,10], [0,0], color='#bbeebb')
                ax.plot([0,0], [0,0], [0,10], color='#bbbbee')
                #new axes
                axis_length = 7
                x = self.rot.apply([1,0,0])
                y = self.rot.apply([0,1,0])
                z = self.rot.apply([0,0,1])
                ax.plot([0,axis_length*x[0]], [0,axis_length*x[1]], [0,axis_length*x[2]], color='r')
                ax.plot([0,axis_length*y[0]], [0,axis_length*y[1]], [0,axis_length*y[2]], color='g')
                ax.plot([0,axis_length*z[0]], [0,axis_length*z[1]], [0,axis_length*z[2]], color='b')
                #testpoint
                if (self.use_testpoint):
                    v = self.testpoint
                    w = self.rot.apply(v)
                    ax.scatter([w[0]], [w[1]], [w[2]], color='black')
                    #gridbox
                    gridcolor='#cccc00'
                    altitudecolor='blue'
                    gridalpha=0.5
                    ax.plot([0,v[0]*x[0]],[0,v[0]*x[1]],[0,v[0]*x[2]],color=gridcolor,linestyle='--',alpha=gridalpha)
                    ax.plot([v[1]*y[0],v[1]*y[0]+v[0]*x[0]],[v[1]*y[1],v[1]*y[1]+v[0]*x[1]],[v[1]*y[2],v[1]*y[2]+v[0]*x[2]],color=gridcolor,linestyle='--',alpha=gridalpha)
                    ax.plot([0,v[1]*y[0]],[0,v[1]*y[1]],[0,v[1]*y[2]],color=gridcolor,linestyle='--',alpha=gridalpha)
                    ax.plot([v[0]*x[0],v[1]*y[0]+v[0]*x[0]],[v[0]*x[1],v[1]*y[1]+v[0]*x[1]],[v[0]*x[2],v[1]*y[2]+v[0]*x[2]],color=gridcolor,linestyle='--',alpha=gridalpha)
                    ax.plot([v[1]*y[0]+v[0]*x[0],v[2]*z[0]+v[1]*y[0]+v[0]*x[0]],[v[1]*y[1]+v[0]*x[1],v[2]*z[1]+v[1]*y[1]+v[0]*x[1]],[v[1]*y[2]+v[0]*x[2],v[2]*z[2]+v[1]*y[2]+v[0]*x[2]],color=altitudecolor,linestyle=':')
                    #shadow
                    xx=[0,v[0]*x[0],v[0]*x[0]+v[1]*y[0],v[1]*y[0]]
                    yy=[0,v[0]*x[1],v[0]*x[1]+v[1]*y[1],v[1]*y[1]]
                    zz=[0,v[0]*x[2],v[0]*x[2]+v[1]*y[2],v[1]*y[2]]
                    verts = [list(zip(xx,yy,zz))]
                    collection = Poly3DCollection(verts,alpha=0.2)
                    collection.set_facecolor(gridcolor)
                    ax.add_collection3d(collection,zs='z')
                
                ax.set_axis_off()
                ax.set_aspect('equal')
                ax.view_init(30,45)
                display(ax.figure)
        refresh_graph(None)
        w_right.on_click(right)
        w_left.on_click(left)
        w_up.on_click(up)
        w_down.on_click(down)
        w_reset.on_click(reset)
        self.is_valid.observe(refresh_graph)
        return vbox

In [111]:
# structure to hold several cards representing rotations, in order
# each card is able to be deleted, and a card can be added to the end
class Book:
    def __init__(self):
        self.cardlist = []
        self.widgetlist = widgets.VBox([])
        self.plus = widgets.Button(description='+',layout=Layout(width="50px"))
        self.widget = widgets.VBox([self.widgetlist,self.plus],layout=Layout(width='550px'))
        self.product = ViewableRot()
        def add_card(sender):
            self.add()
        self.plus.on_click(add_card)
        self.add()
    def size(self):
        return len(self.cardlist)
    def display(self):
        return self.widget
    def displayProduct(self):
        return self.product.display()
    def getProduct(self):
        return self.product.get_rot()
    def add(self):
        # update book
        if (self.size() > 0):
            self.cardlist[-1].enableDelete()
        # create new card
        newcard = Card(self)
        newcard.enableDelete()
        def update(sender):
            self.updateProduct()
        # link card to book
        newcard.link(update)
        # add card to book
        self.cardlist = self.cardlist + [newcard]
        self.widgetlist.children = list(self.widgetlist.children) + [newcard.display()]
    def remove(self,card):
        # find card index
        index = 0
        while (self.cardlist[index] != card):
            index = index + 1
        # don't delete all cards
        if (index > 0 or self.size() > 1):
            # remove card and its widget
            self.cardlist[index].close()
            del self.cardlist[index]
            newwidgetlist = list(self.widgetlist.children)
            del newwidgetlist[index]
            self.widgetlist.children = newwidgetlist
            # update book
            if (self.size() == 1):
                self.cardlist[0].disableDelete()
            self.updateProduct()
    def updateProduct(self):
        rot = R.from_rotvec([0,0,0])
        for card in self.cardlist:
            cardrot = card.rot.get_rot()
            rot = cardrot * rot
        self.product.set_rot(rot)

# structure to hold a rotation inside a book
# Input widget allows for the rotation to be inputted
# Delete Button allows the card to be removed from its book
class Card:
    def __init__(self,book,rot=None):
        self.book = book
        self.rot = rot
        if rot is None:
            self.rot = ViewableRot()
        self.minus = widgets.Button(disabled=True,description='-',layout=Layout(width="50px"))
        self.input = self.rot.input_rot()
        self.widget = widgets.HBox([self.input, self.minus])
        def close_rot(sender):
            self.book.remove(self)
        self.minus.on_click(close_rot)
    def display(self):
        return self.widget
    def close(self):
        self.minus.close()
        self.input.close()
        self.widget.close()
    def disableDelete(self):
        self.minus.disabled=True
    def enableDelete(self):
        self.minus.disabled=False
    def link(self,f):
        # updates book when rotation is changed
        self.rot.is_valid.observe(f)

In [112]:
Title = widgets.Label(value=r'\(\Huge Coordinate \space System \space Viewer\)',layout=Layout(height='50px'))
TitleHBox = widgets.HBox([Title])
TitleHBox.layout.justify_content='center'
############################################################################################
CSTop_Title = widgets.Label(value=r'\(\Large Rotation \space Viewer\)')

rot1 = ViewableRot(R.from_rotvec([0,0,0]))
rot2 = ViewableRot(R.from_rotvec([0,0,0]))
rot3 = ViewableRot(R.from_rotvec([0,0,0]))

def f(sender):
    r1 = rot1.get_rot()
    r2 = rot2.get_rot()
    rot3.set_rot(r2 * r1.inv())
    
rot1.is_valid.observe(f)
rot2.is_valid.observe(f)

CS2_input = rot2.input_rot()
CS1_disp = rot1.display()
CS2_disp = rot2.display()
CS3_disp = rot3.display()
CS_input_and_display = widgets.HBox([CS2_input,CS3_disp])
CS_input_and_display.layout.justify_content='space-around'
CS1_graph = rot1.graph()
CS2_graph = rot2.graph()
CS_graphs = widgets.HBox([CS1_graph,CS2_graph])


CSTop = widgets.VBox([CSTop_Title,CS_input_and_display,CS_graphs])
############################################################################################
CSBottom_Title = widgets.Label(value=r'\(\Large Rotation \space Multiplier\)')
tb = Book()
tbd = tb.display()
tbp = tb.displayProduct()
tbpg = tb.product.graph()
CSBottomR = widgets.VBox([tbp,tbpg])
CSBottomB = widgets.HBox([tbd,CSBottomR])

CSBottom = widgets.VBox([CSBottom_Title,CSBottomB])
#############################################################################################

CSViewer = widgets.VBox([TitleHBox, CSTop, CSBottom])
#CSViewer.layout.align_items = 'center'
CSViewer